# Modelo de impacto dos atributos

Após extrair o histórico de atualizações do jogo, foi necessário classificar manualmente as alterações já conhecidas. Essa classificação se baseia na questão do _impacto_ de cada atributo alterado para o personagem: impacto _direto_ (`1`) é quando a presença daquela alteração ou o aumento daquele atributo incorre numa melhoria para o personagem; impacto _inverso_ (`-1`) é quando a remoção de uma característica ou a redução de um dado atributo incorre numa melhoria para o personagem. Além disso, também foram classificadas as alterações que são irrelevantes (`0` ou `-2`).

Para não ter que fazer essa classificação manualmente sempre, podemos criar um modelo que classifica frases de notas de atualização entre alterações de impacto direto, inverso ou irrelevante.

In [2]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

import pandas as pd

In [3]:
raw_df = pd.read_csv("data/impact.csv")
raw_df

,champion,about,text,impact
0,Ahri,Fox-Fire,Mana cost increased to 30 from 25.,-1.0
1,Ahri,Fox-Fire,Base damage reduced to 50 / 75 / 100 / 125 / 1...,1.0
2,Ahri,Charm,Mana cost increased to 60 from 50.,-1.0
3,Caitlyn,Piltover Peacemaker,Bug Fix: Now properly deals damage even if the...,0.0
4,Elise,General,Updated visual effects.,-2.0
...,...,...,...,...
22622,Teemo,NaN,Fixed attack timing bugs.,0.0
22623,Tristana,NaN,Corrected tooltip typos for Rapid Fire.,0.0
22624,Warwick,NaN,Corrected tooltip typos for Eternal Thirst.,0.0
22625,Warwick,NaN,Fixed tooltip functionality for Hungering Str...,0.0


Há uma pequena distinção entre `0` - indicando correção de bugs - e `-2` - indicando alterações apenas cosméticas ou indefinidas. Por motivos de simplicidade podemos tratar ambos como "irrelevante".

In [5]:
df = raw_df.copy()
df['impact'] = df['impact'].map(lambda x: 0 if x == -2 else x)
df

,champion,about,text,impact
0,Ahri,Fox-Fire,Mana cost increased to 30 from 25.,-1.0
1,Ahri,Fox-Fire,Base damage reduced to 50 / 75 / 100 / 125 / 1...,1.0
2,Ahri,Charm,Mana cost increased to 60 from 50.,-1.0
3,Caitlyn,Piltover Peacemaker,Bug Fix: Now properly deals damage even if the...,0.0
4,Elise,General,Updated visual effects.,0.0
...,...,...,...,...
22622,Teemo,NaN,Fixed attack timing bugs.,0.0
22623,Tristana,NaN,Corrected tooltip typos for Rapid Fire.,0.0
22624,Warwick,NaN,Corrected tooltip typos for Eternal Thirst.,0.0
22625,Warwick,NaN,Fixed tooltip functionality for Hungering Str...,0.0


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score

from steps.transform import PatchNoteTransformer

X, y = df, df['impact']
model = Pipeline([
    ('patch', PatchNoteTransformer()),
    ('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier()),
])

In [37]:
evaluate = False

In [38]:
if evaluate:
    score = cross_val_score(model, X, y, cv=10)
    print(f"Cross val score: {score.mean():.2f} (+/- {score.std() * 2:.2f})")
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)
    model.fit(train_x, train_y)
    predicted = model.predict(test_x)
    print(metrics.classification_report(test_y, predicted))


In [14]:
import joblib
import os

if not os.path.exists('model'):
  os.mkdir('model')

model.fit(X, y)
joblib.dump(model, 'model/impact_classifier.pickle')

['model/impact_classifier.pickle']

In [15]:
predicted = model.predict(X)
print(metrics.classification_report(y, predicted))

              precision    recall  f1-score   support

        -1.0       1.00      1.00      1.00      3214
         0.0       1.00      1.00      1.00      6590
         1.0       1.00      1.00      1.00     12823

    accuracy                           1.00     22627
   macro avg       1.00      1.00      1.00     22627
weighted avg       1.00      1.00      1.00     22627

